In [2]:
%load_ext nvcc4jupyter

from nvcc4jupyter import set_defaults
set_defaults(compiler_args='-arch=sm_100a -Xptxas=-v -O0')


Source files will be saved in "/tmp/tmp5pdr6r__".


In [3]:
%%cuda
#include <mma.h>
#include <stdio.h>
#include <cuda_runtime.h> 
#include <cuda.h> 
#include <cuda_fp16.h>

constexpr int N_warps = 1; 
constexpr int atom_M = 16; 
constexpr int atom_N = 16; 
constexpr int atom_K = 16;

size_t tensor_size = atom_M * atom_K * sizeof(half);
size_t C_size = atom_M * atom_N * sizeof(float);

using namespace nvcuda;

__global__ void wmma_ker(half *a, half *b, float *c) {
   wmma::fragment<wmma::matrix_a, 16, 16, 16, half, wmma::col_major> a_frag;
   wmma::fragment<wmma::matrix_b, 16, 16, 16, half, wmma::row_major> b_frag;
   wmma::fragment<wmma::accumulator, 16, 16, 16, float> c_frag;

   wmma::fill_fragment(c_frag, 0.0f);

   wmma::load_matrix_sync(a_frag, a, 16);
   wmma::load_matrix_sync(b_frag, b, 16);

   wmma::mma_sync(c_frag, a_frag, b_frag, c_frag);

   wmma::store_matrix_sync(c, c_frag, 16, wmma::mem_row_major);
}

int main() {
  float *C, *C_d;
  half *A, *A_d, *B, *B_d;

  cudaHostAlloc(&A, tensor_size, cudaHostAllocDefault); 
  cudaHostAlloc(&B, tensor_size, cudaHostAllocDefault); 
  cudaHostAlloc(&C, C_size, cudaHostAllocDefault); 
  
  cudaMalloc(&A_d, tensor_size); 
  cudaMalloc(&B_d, tensor_size); 
  cudaMalloc(&C_d, C_size);

  for (int i = 0; i < atom_M * atom_K; i++) {
      A[i] = (half)1.0f;
      B[i] = (half)1.0f;
  }

  cudaMemcpy(A_d, A, tensor_size, cudaMemcpyHostToDevice);
  cudaMemcpy(B_d, B, tensor_size, cudaMemcpyHostToDevice);
  cudaMemset(C_d, 0, C_size);

  cudaEvent_t start, stop;
  cudaEventCreate(&start);
  cudaEventCreate(&stop);

  cudaEventRecord(start);
  wmma_ker<<<1, 32>>>(A_d, B_d, C_d);
  cudaEventRecord(stop);

  cudaEventSynchronize(stop);
  
  float milliseconds = 0;
  cudaEventElapsedTime(&milliseconds, start, stop);

  cudaMemcpy(C, C_d, C_size, cudaMemcpyDeviceToHost);

  double ops = 2.0 * atom_M * atom_N * atom_K; 
  double gflops = (ops / 1e9) / (milliseconds / 1000.0f);

  printf("Time: %f ms\n", milliseconds);
  printf("Performance: %f GFLOPS\n", gflops);
  printf("Check C[0]: %f\n", C[0]);

  cudaFreeHost(A); cudaFreeHost(B); cudaFreeHost(C);
  cudaFree(A_d); cudaFree(B_d); cudaFree(C_d);
  
  return 0;
}

Time: 1.613792 ms
Performance: 0.005076 GFLOPS
Check C[0]: 16.000000



In [4]:
%%cuda
#include <mma.h>
#include <stdio.h>
#include <cuda_runtime.h> 
#include <cuda.h> 
#include <cuda_fp16.h>

constexpr int N_warps = 1; 
constexpr int atom_M = 16; 
constexpr int atom_N = 16; 
constexpr int atom_K = 16;

size_t tensor_size = atom_M * atom_K * sizeof(half);
size_t C_size = atom_M * atom_N * sizeof(float);

using namespace nvcuda;

__global__ void wmma_ker(half *a, half *b, float *c) {
  wmma::fragment<wmma::matrix_a, 16, 16, 16, half, wmma::col_major> a_frag;
  wmma::fragment<wmma::matrix_b, 16, 16, 16, half, wmma::row_major> b_frag;
  wmma::fragment<wmma::accumulator, 16, 16, 16, float> c_frag;
  
  __shared__ alignas(256) half As[32*32];
  __shared__ alignas(256) half Bs[32*32];
  int t = threadIdx.x; 
  int t_col = t % 16; 
  int t_row = t / 16;
  
  for (int i = 0; i < 16; i+= 2)
  {
    int idx = (t_row + i)*16 + t_col; 
    As[(3*16*16) + idx] = a[idx];
    Bs[(3*16*16) + idx] = b[idx];
    
  }

  __syncthreads();

  wmma::fill_fragment(c_frag, 0.0f);

  wmma::load_matrix_sync(a_frag, &As[(3*16*16)], 16);
  wmma::load_matrix_sync(b_frag, &Bs[(3*16*16)], 16);

  wmma::mma_sync(c_frag, a_frag, b_frag, c_frag);

  wmma::store_matrix_sync(c, c_frag, 16, wmma::mem_row_major);
  
}

int main() {
  float *C, *C_d;
  half *A, *A_d, *B, *B_d;

  cudaHostAlloc(&A, tensor_size, cudaHostAllocDefault); 
  cudaHostAlloc(&B, tensor_size, cudaHostAllocDefault); 
  cudaHostAlloc(&C, C_size, cudaHostAllocDefault); 

  cudaMalloc(&A_d, tensor_size); 
  cudaMalloc(&B_d, tensor_size); 
  cudaMalloc(&C_d, C_size);

  for (int i = 0; i < atom_M * atom_K; i++) {
      A[i] = (half)1.0f;
      B[i] = (half)1.0f;
  }

  cudaMemcpy(A_d, A, tensor_size, cudaMemcpyHostToDevice);
  cudaMemcpy(B_d, B, tensor_size, cudaMemcpyHostToDevice);
  cudaMemset(C_d, 0, C_size);

  cudaEvent_t start, stop;
  cudaEventCreate(&start);
  cudaEventCreate(&stop);

  cudaEventRecord(start);
  wmma_ker<<<1, 32>>>(A_d, B_d, C_d);
  cudaEventRecord(stop);

  cudaEventSynchronize(stop);

  float milliseconds = 0;
  cudaEventElapsedTime(&milliseconds, start, stop);

  cudaMemcpy(C, C_d, C_size, cudaMemcpyDeviceToHost);

  double ops = 2.0 * atom_M * atom_N * atom_K; 
  double gflops = (ops / 1e9) / (milliseconds / 1000.0f);

  printf("Time: %f ms\n", milliseconds);
  printf("Performance: %f GFLOPS\n", gflops);
  printf("Check C[0]: %f\n", C[0]);

  cudaFreeHost(A); cudaFreeHost(B); cudaFreeHost(C);
  cudaFree(A_d); cudaFree(B_d); cudaFree(C_d);

  return 0;
}

Time: 0.212352 ms
Performance: 0.038577 GFLOPS
Check C[0]: 16.000000



In [12]:
%%cuda
#include <iostream>
#include <vector>
#include <random>
#include <cmath>
#include <cassert>
#include <cstdio>
#include <cuda_runtime.h>
#include <cuda_bf16.h>
#include <cudaTypedefs.h> 
#include <cuda.h>

#define CUDA_CHECK(call) \
    do { \
        cudaError_t err = call; \
        if (err != cudaSuccess) { \
            fprintf(stderr, "CUDA error at %s:%d code=%d(%s) \"%s\" \n", \
                __FILE__, __LINE__, err, cudaGetErrorString(err), #call); \
            exit(EXIT_FAILURE); \
        } \
    } while (0)

PFN_cuTensorMapEncodeTiled_v12000 get_cuTensorMapEncodeTiled() {
  cudaDriverEntryPointQueryResult driver_status;
  void* cuTensorMapEncodeTiled_ptr = nullptr;
  CUDA_CHECK(cudaGetDriverEntryPointByVersion("cuTensorMapEncodeTiled", &cuTensorMapEncodeTiled_ptr, 12000, cudaEnableDefault, &driver_status));
  assert(driver_status == cudaDriverEntryPointSuccess);
  return reinterpret_cast<PFN_cuTensorMapEncodeTiled_v12000>(cuTensorMapEncodeTiled_ptr);
}

constexpr int N_warps = 32; 
constexpr int atom_M = 16; 
constexpr int atom_N = 16; 
constexpr int atom_K = 16;
constexpr int tiled_M = 4;
constexpr int tiled_N = 8;
constexpr int tiled_K = 8;

static_assert(tiled_M * tiled_N == N_warps, "Error");

constexpr int M = atom_M * tiled_M; 
constexpr int N = atom_N * tiled_N; 
constexpr int K = atom_K * tiled_K; 

constexpr uint32_t rank = 2;
uint64_t A_size[rank] = {K,M}; 
uint64_t B_size[rank] = {N,K}; 
uint64_t A_stride[rank-1] = {K*sizeof(__nv_bfloat16)}; 
uint64_t B_stride[rank-1] = {N*sizeof(__nv_bfloat16)};
uint32_t A_smem_size[rank] = {atom_K, atom_M}; 
uint32_t B_smem_size[rank] = {atom_N, atom_K}; 
uint32_t elem_stride[rank] = {1,1};

auto pfn_cuTensorMapEncodeTiled = get_cuTensorMapEncodeTiled();

__global__ void wmma_ker(const __grid_constant__ CUtensorMap tensorMapA, 
                         const __grid_constant__ CUtensorMap tensorMapB, 
                         float *c) 
{
    extern __shared__ uint8_t raw_smem[];
}

void cpu_matmul(const std::vector<float>& h_A, const std::vector<float>& h_B, std::vector<float>& h_C_ref) {
    for (int i = 0; i < M; ++i) {
        for (int j = 0; j < N; ++j) {
            float sum = 0.0f;
            for (int k = 0; k < K; ++k) {
                sum += h_A[i * K + k] * h_B[k * N + j];
            }
            h_C_ref[i * N + j] = sum;
        }
    }
}

int main() 
{
  size_t A_len = M * K;
  size_t B_len = K * N;
  size_t C_len = M * N;

  size_t A_bytes = sizeof(__nv_bfloat16) * A_len;
  size_t B_bytes = sizeof(__nv_bfloat16) * B_len;
  size_t C_bytes = sizeof(float) * C_len;

  std::vector<float> h_A_float(A_len);
  std::vector<float> h_B_float(B_len);
  std::vector<float> h_C_ref(C_len);
  std::vector<float> h_C_gpu(C_len);

  std::mt19937 gen(1337);
  std::uniform_real_distribution<float> dis(-1.0f, 1.0f);

  for(auto &x : h_A_float) x = dis(gen);
  for(auto &x : h_B_float) x = dis(gen);

  std::vector<__nv_bfloat16> h_A_bf16(A_len);
  std::vector<__nv_bfloat16> h_B_bf16(B_len);

  for(size_t i = 0; i < A_len; ++i) h_A_bf16[i] = __float2bfloat16(h_A_float[i]);
  for(size_t i = 0; i < B_len; ++i) h_B_bf16[i] = __float2bfloat16(h_B_float[i]);

  __nv_bfloat16 *d_A, *d_B;
  float *d_C;
  
  cudaMalloc(&d_A, A_bytes);
  cudaMalloc(&d_B, B_bytes);
  cudaMalloc(&d_C, C_bytes);

  cudaMemcpy(d_A, h_A_bf16.data(), A_bytes, cudaMemcpyHostToDevice);
  cudaMemcpy(d_B, h_B_bf16.data(), B_bytes, cudaMemcpyHostToDevice);
  cudaMemset(d_C, 0, C_bytes);

  int threads_per_warp = 32;
  int block_size = N_warps * threads_per_warp;
  size_t smem_size = A_bytes + B_bytes + C_bytes;

  cudaFuncSetAttribute(wmma_ker, cudaFuncAttributeMaxDynamicSharedMemorySize, smem_size);

  CUtensorMap tMap_A{};
  CUtensorMap tMap_B{}; 
  
  CUresult res_A = pfn_cuTensorMapEncodeTiled(
    &tMap_A, 
    CUtensorMapDataType::CU_TENSOR_MAP_DATA_TYPE_BFLOAT16,
    rank,
    d_A, 
    A_size,
    A_stride,
    A_smem_size,
    elem_stride,
    CUtensorMapInterleave::CU_TENSOR_MAP_INTERLEAVE_NONE,
    CUtensorMapSwizzle::CU_TENSOR_MAP_SWIZZLE_NONE,
    CUtensorMapL2promotion::CU_TENSOR_MAP_L2_PROMOTION_NONE,
    CUtensorMapFloatOOBfill::CU_TENSOR_MAP_FLOAT_OOB_FILL_NONE
  );
  assert(res_A == CUDA_SUCCESS);
  
  CUresult res_B = pfn_cuTensorMapEncodeTiled(
    &tMap_B, 
    CUtensorMapDataType::CU_TENSOR_MAP_DATA_TYPE_BFLOAT16,
    rank,
    d_B, 
    B_size,
    B_stride,
    B_smem_size,
    elem_stride,
    CUtensorMapInterleave::CU_TENSOR_MAP_INTERLEAVE_NONE,
    CUtensorMapSwizzle::CU_TENSOR_MAP_SWIZZLE_NONE,
    CUtensorMapL2promotion::CU_TENSOR_MAP_L2_PROMOTION_NONE,
    CUtensorMapFloatOOBfill::CU_TENSOR_MAP_FLOAT_OOB_FILL_NONE
  );
  assert(res_B == CUDA_SUCCESS);

  cudaEvent_t start, stop;
  cudaEventCreate(&start);
  cudaEventCreate(&stop);

  wmma_ker<<<1, block_size, smem_size>>>(tMap_A, tMap_B, d_C);

  cudaEventRecord(start);
  wmma_ker<<<1, block_size, smem_size>>>(tMap_A, tMap_B, d_C);
  cudaEventRecord(stop);

  cudaEventSynchronize(stop);
  float milliseconds = 0;
  cudaEventElapsedTime(&milliseconds, start, stop);

  cudaMemcpy(h_C_gpu.data(), d_C, C_bytes, cudaMemcpyDeviceToHost);

  cpu_matmul(h_A_float, h_B_float, h_C_ref);

  float max_error = 0.0f;
  for(size_t i = 0; i < C_len; ++i) {
      float diff = std::abs(h_C_ref[i] - h_C_gpu[i]);
      if(diff > max_error) max_error = diff;
  }

  double flops = 2.0 * M * N * K;
  double gflops = (flops * 1e-9) / (milliseconds / 1000.0);

  printf("M: %d, N: %d, K: %d\n", M, N, K);
  printf("Time: %.4f ms\n", milliseconds);
  printf("Performance: %.4f GFLOPS\n", gflops);
  printf("Max Error: %f\n", max_error);

  cudaFree(d_A); cudaFree(d_B); cudaFree(d_C);
  cudaEventDestroy(start); cudaEventDestroy(stop);

  return 0;
}

M: 64, N: 128, K: 128
Time: 0.0046 ms
Performance: 455.1111 GFLOPS
Max Error: 15.367403

